In [20]:
from pyspark.sql import SparkSession

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf 
from pyspark.sql import Window 
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, BooleanType, LongType, FloatType, IntegerType 


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark = SparkSession.builder.appName("Spark Project").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
data = "s3n://stock-market-test/AARTIIND__EQ__NSE__NSE__MINUTE.csv"
df = spark.read.csv(data,  header='true')
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+-----+-----+-----+------+
|           timestamp| open| high|  low|close|volume|
+--------------------+-----+-----+-----+-----+------+
|2017-01-02 09:15:...|340.0|340.0|340.0|340.0|  11.0|
|2017-01-02 09:16:...|340.0|340.0|340.0|340.0|   0.0|
|2017-01-02 09:17:...|340.0|340.0|340.0|340.0|   0.0|
|2017-01-02 09:18:...|340.0|343.7|340.0|343.7|   1.0|
|2017-01-02 09:19:...|343.7|343.7|343.7|343.7|   1.0|
+--------------------+-----+-----+-----+-----+------+
only showing top 5 rows

In [131]:
#Resampling on Daily Basis
df.createOrReplaceTempView('data');
query2 = spark.sql("SELECT DATE(timestamp) as Date,FIRST(open) as OPEN,MIN(low) as LOW,MAX(high) as HIGH,LAST(close) as CLOSE,SUM(volume) as VOLUME from data group by DATE(timestamp) order by Date")
query2.show(50)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+------+------+------+--------+
|      Date|  OPEN|   LOW|  HIGH| CLOSE|  VOLUME|
+----------+------+------+------+------+--------+
|2017-01-02| 340.0| 337.3|348.95| 346.0|  8372.0|
|2017-01-03| 349.5| 341.8| 355.0| 350.0| 20826.0|
|2017-01-04| 353.3| 341.5|355.65| 350.0|131883.0|
|2017-01-05|352.35|349.55|354.95| 351.5|  8818.0|
|2017-01-06| 352.0| 349.0| 354.9| 349.1|  8128.0|
|2017-01-09|351.75| 348.0| 352.0| 348.5|  6310.0|
|2017-01-10| 350.0| 348.0|352.25| 350.5| 10186.0|
|2017-01-11| 350.5|346.75|351.35|351.25| 17155.0|
|2017-01-12| 350.5|349.05|351.95| 350.0|  8469.0|
|2017-01-13| 348.1| 348.1| 354.0|351.95| 12967.0|
|2017-01-16| 351.9| 350.0| 355.0|353.95|  8673.0|
|2017-01-17|353.55| 352.1| 354.5| 354.0|  7386.0|
|2017-01-18| 355.0| 352.5|360.75| 356.5| 27663.0|
|2017-01-19| 355.2|350.05| 359.7| 352.5| 16641.0|
|2017-01-20| 355.0| 342.5| 355.0|343.25| 28588.0|
|2017-01-23| 340.6|336.55|353.35| 353.0| 15594.0|
|2017-01-24| 350.0| 346.5|353.85| 346.5| 20011.0|


In [160]:
#Returns on a Stock
df.createOrReplaceTempView('data');
query3 = spark.sql("SELECT LAST(close)/FIRST(close)* 100 as Percentage_Return from data")
query3.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|Percentage_Return|
+-----------------+
|370.5882352941177|
+-----------------+

In [174]:
#Amount Traded
df.createOrReplaceTempView('data');
query4 = spark.sql("SELECT DATE(timestamp) ,ROUND(LAST(close)-FIRST(open),2) as volatility from data group by DATE(timestamp) order by volatility DESC")
query4.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+
| timestamp|volatility|
+----------+----------+
|2020-04-27|      95.0|
|2020-03-13|      88.8|
|2020-01-29|      88.0|
|2018-01-05|      85.0|
|2020-04-28|      68.0|
|2020-08-20|      57.1|
|2017-03-02|     55.35|
|2020-11-05|      54.0|
|2020-11-27|     51.35|
|2019-10-18|      48.5|
|2019-05-30|      45.0|
|2020-03-31|      44.0|
|2018-10-03|      43.0|
|2020-02-20|     42.15|
|2019-09-20|      41.2|
|2020-12-07|     41.15|
|2020-08-19|     40.95|
|2018-08-30|     39.75|
|2020-01-28|     39.15|
|2019-02-04|      38.5|
+----------+----------+
only showing top 20 rows

In [159]:
#Daily Volume on trading
df.createOrReplaceTempView('data');
query1 = spark.sql("SELECT DATE(timestamp) as Date,SUM(volume) as Volume from data group by DATE(timestamp) order by Date")
query1.show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+
|      Date|  Volume|
+----------+--------+
|2017-01-02|  8372.0|
|2017-01-03| 20826.0|
|2017-01-04|131883.0|
|2017-01-05|  8818.0|
|2017-01-06|  8128.0|
|2017-01-09|  6310.0|
|2017-01-10| 10186.0|
|2017-01-11| 17155.0|
|2017-01-12|  8469.0|
|2017-01-13| 12967.0|
|2017-01-16|  8673.0|
|2017-01-17|  7386.0|
|2017-01-18| 27663.0|
|2017-01-19| 16641.0|
|2017-01-20| 28588.0|
|2017-01-23| 15594.0|
|2017-01-24| 20011.0|
|2017-01-25| 62783.0|
|2017-01-27| 45212.0|
|2017-01-30| 43072.0|
+----------+--------+
only showing top 20 rows

In [153]:
#Resampling on Daily Basis
df.createOrReplaceTempView('data');
query2 = spark.sql("SELECT LAST(close) from data")
query2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+
|last(close)|
+-----------+
|     1260.0|
+-----------+

In [77]:
df.summary()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[summary: string, timestamp: string, open: string, high: string, low: string, close: string, volume: string]

In [79]:
df1 = df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

aws-cfn-bootstrap (2.0)
beautifulsoup4 (4.9.3)
boto (2.49.0)
click (8.1.1)
Cython (0.29.32)
docutils (0.14)
jmespath (1.0.0)
joblib (1.1.0)
lockfile (0.11.0)
lxml (4.8.0)
mysqlclient (1.4.2)
nltk (3.7)
nose (1.3.4)
numpy (1.20.0)
pip (9.0.1)
py-dateutil (2.2)
pystache (0.5.4)
python-daemon (2.2.3)
python-dateutil (2.8.2)
python37-sagemaker-pyspark (1.4.1)
pytz (2022.1)
PyYAML (5.4.1)
regex (2021.11.10)
setuptools (28.8.0)
simplejson (3.2.0)
six (1.13.0)
tqdm (4.63.1)
wheel (0.29.0)
windmill (1.6)

DEPRECATION: The default format will switch to columns in the future. You can use --format=(legacy|columns) (or define a format=(legacy|columns) in your pip.conf under the [list] section) to disable this warning.
You are using pip version 9.0.1, however version 22.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [81]:
df= df.withColumn("open",df.open.cast('float'))
df= df.withColumn("high",df.high.cast('float'))
df= df.withColumn("close",df.close.cast('float'))
df= df.withColumn("volume",df.volume.cast('float'))
df= df.withColumn("low",df.low.cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
df= df.withColumn("timestamp",to_timestamp('timestamp'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'to_timestamp' is not defined
Traceback (most recent call last):
NameError: name 'to_timestamp' is not defined



In [85]:
df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[timestamp: string, open: float, high: float, low: float, close: float, volume: float]

In [84]:
df1['timestamp']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Column<b'timestamp'>

In [87]:
import pyspark.sql.functions as F

test_df= df
test_df_date = test_df.withColumn('timestamp', F.to_date(test_df['timestamp']))

test_df_date.orderBy('timestamp').show() 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+------+------+------+------+
| timestamp|  open|  high|   low| close|volume|
+----------+------+------+------+------+------+
|2017-01-02|339.85|339.85|339.45|339.45|   5.0|
|2017-01-02| 338.0| 338.0| 337.5| 337.5|  20.0|
|2017-01-02|339.85|339.85|339.85|339.85|   0.0|
|2017-01-02| 341.0| 341.0| 339.0| 339.0|   7.0|
|2017-01-02| 337.5| 338.0| 337.5| 338.0|  68.0|
|2017-01-02| 339.0| 339.0| 339.0| 339.0| 102.0|
|2017-01-02| 339.0|339.85| 339.0|339.85|   5.0|
|2017-01-02| 340.0| 340.0| 340.0| 340.0|   0.0|
|2017-01-02| 341.0| 341.0| 341.0| 341.0| 932.0|
|2017-01-02| 338.0|338.95| 338.0|338.95|   4.0|
|2017-01-02| 337.3| 338.5| 337.3| 337.5| 131.0|
|2017-01-02| 337.5| 339.5| 337.5| 339.5|  62.0|
|2017-01-02| 339.5|339.95| 339.5|339.95|  10.0|
|2017-01-02| 339.0| 339.0| 339.0| 339.0|   1.0|
|2017-01-02| 339.0| 339.0| 339.0| 339.0|   5.0|
|2017-01-02| 340.0| 340.0| 340.0| 340.0|  11.0|
|2017-01-02| 340.0| 340.0| 340.0| 340.0|   0.0|
|2017-01-02| 340.0| 343.7| 340.0| 343.7|

In [89]:
test_df_date.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+------+------+------+------+
| timestamp|  open|  high|   low| close|volume|
+----------+------+------+------+------+------+
|2017-01-02| 340.0| 340.0| 340.0| 340.0|  11.0|
|2017-01-02| 340.0| 340.0| 340.0| 340.0|   0.0|
|2017-01-02| 340.0| 340.0| 340.0| 340.0|   0.0|
|2017-01-02| 340.0| 343.7| 340.0| 343.7|   1.0|
|2017-01-02| 343.7| 343.7| 343.7| 343.7|   1.0|
|2017-01-02| 341.0| 341.0| 341.0| 341.0| 932.0|
|2017-01-02| 341.0| 341.0| 339.0| 339.0|   7.0|
|2017-01-02| 339.0|339.15| 338.0| 338.0|  26.0|
|2017-01-02| 338.0|338.95| 338.0|338.95|   4.0|
|2017-01-02| 337.3| 338.5| 337.3| 337.5| 131.0|
|2017-01-02| 337.5| 338.0| 337.5| 338.0|  68.0|
|2017-01-02| 338.0| 338.0| 337.5| 337.5|  20.0|
|2017-01-02| 337.5| 339.5| 337.5| 339.5|  62.0|
|2017-01-02| 339.5|339.95| 339.5|339.95|  10.0|
|2017-01-02| 339.0| 339.0| 339.0| 339.0| 102.0|
|2017-01-02| 339.0| 339.0| 339.0| 339.0|   1.0|
|2017-01-02| 339.0| 339.0| 339.0| 339.0|   5.0|
|2017-01-02| 339.0|339.85| 339.0|339.85|

In [115]:
query2 = spark.sql("Select timestamp, SUM(volume) from data groupby timestamp")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
"\nextraneous input 'timestamp' expecting <EOF>(line 1, pos 48)\n\n== SQL ==\nSelect timestamp, SUM(volume) from data groupby timestamp\n------------------------------------------------^^^\n"
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 767, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 73, in deco
    raise ParseException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.ParseException: "\nextraneous input 'timestamp' expecting <EOF>(line 1, pos 48)\n\n== SQL ==\nSelect timestamp, SUM(volume) from data groupby timestamp\n------------------------------------------------^^^\n"



In [111]:
data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3n://stock-market-test/AARTIIND__EQ__NSE__NSE__MINUTE.csv'

In [117]:
next_start_time = F.lead('timestamp').over(Window.partitionBy('volume').orderBy('timestamp'))
end_time = F.when(next_start_time.isNull(),
                  F.col('timestamp')
           ).otherwise(
                  F.date_sub(next_start_time, 1)
           )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'Column' object is not callable
Traceback (most recent call last):
TypeError: 'Column' object is not callable

